In [3]:
import pandas as pd

# "patient_id exam_id spot[1-14] frame_number score"
# load patients data in a dataframe from images folder
import glob
images_paths = glob.glob("images/*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[7:-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(int)
images_df["frame_number"] = images_df["frame_number"].astype(int)
images_df["spot"] = images_df["spot"].astype(int)
images_df["patient_id"] = images_df["patient_id"].astype(int)
images_df["exam_id"] = images_df["exam_id"].astype(int)
images_df.tail()


,patient_id,exam_id,spot,frame_number,score
397,1069,1158,7,117,1
398,1069,1158,7,118,1
399,1069,1158,7,119,1
400,1069,1158,7,120,1
401,1069,1158,7,121,1


In [4]:
# prepare the patients in each set
# get the set of patients id
patients_id = set(images_df["patient_id"])
print(len(patients_id))


1
